**Importing the Dependencies**

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [12]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [13]:
# Last 5 rows of the dataset

credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
95157,65189,1.142364,-1.090177,0.511068,-0.792356,-1.160044,0.134166,-0.997184,0.236721,-0.583865,...,0.399798,0.815983,-0.172587,-0.298962,0.310993,-0.077793,0.011223,0.019004,106.00,0.0
95158,65190,-2.145283,1.280406,0.014577,-2.003358,1.479294,4.673049,-2.008023,-2.933663,0.036717,...,0.276433,-0.038006,0.085076,1.049870,0.489570,1.045371,-0.363337,-0.222526,34.61,0.0
95159,65190,-3.715715,3.870511,-1.525809,0.082535,-0.244009,-0.901579,0.708830,0.070491,2.349423,...,-0.327180,0.573451,0.266379,0.040564,-0.175983,-0.494220,0.257349,-0.309196,0.89,0.0
95160,65190,-5.164795,4.510526,-0.994499,-1.110853,-0.913228,-0.889076,0.373572,0.361552,3.841062,...,-0.908623,-1.154210,0.300341,-0.102776,0.817800,0.201861,2.384092,1.576142,7.18,0.0
95161,65191,-1.430966,1.192670,1.237388,1.074059,-0.997949,0.687186,-1.045570,1.012203,0.095426,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95162 entries, 0 to 95161
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    95162 non-null  int64  
 1   V1      95162 non-null  float64
 2   V2      95162 non-null  float64
 3   V3      95162 non-null  float64
 4   V4      95162 non-null  float64
 5   V5      95162 non-null  float64
 6   V6      95162 non-null  float64
 7   V7      95162 non-null  float64
 8   V8      95162 non-null  float64
 9   V9      95162 non-null  float64
 10  V10     95162 non-null  float64
 11  V11     95162 non-null  float64
 12  V12     95162 non-null  float64
 13  V13     95161 non-null  float64
 14  V14     95161 non-null  float64
 15  V15     95161 non-null  float64
 16  V16     95161 non-null  float64
 17  V17     95161 non-null  float64
 18  V18     95161 non-null  float64
 19  V19     95161 non-null  float64
 20  V20     95161 non-null  float64
 21  V21     95161 non-null  float64
 22

In [16]:
#Checking the number of missing values in each column

credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [17]:
credit_card_data['Class'].value_counts()

,count
Class,
0.0,94944
1.0,217


This Dataset is highly imbalance


0 is Normal Transcation

1 is fraudulent Transaction

In [18]:
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [19]:
print(legit.shape)
print(fraud.shape)

(94944, 31)
(217, 31)


In [20]:
legit.Amount.describe()

,Amount
count,94944.000000
mean,98.685374
std,267.432993
min,0.000000
25%,7.600000
50%,26.720000
75%,89.752500
max,19656.530000


In [22]:
fraud.Amount.describe()

,Amount
count,217.000000
mean,109.784286
std,243.927116
min,0.000000
25%,1.000000
50%,7.580000
75%,99.990000
max,1809.680000


In [23]:
credit_card_data.groupby('Class').mean() #compare the values for both transactions


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,41232.783620,-0.25064,-0.047145,0.696491,0.15125,-0.270069,0.097914,-0.093292,0.048787,-0.032333,...,0.043401,-0.031938,-0.107340,-0.037071,0.009933,0.13190,0.026607,-0.000819,0.001332,98.685374
1.0,35870.354839,-6.16618,4.217965,-8.087819,4.99603,-4.444345,-1.845983,-6.417590,2.796291,-2.953535,...,0.354728,0.731128,-0.127758,-0.247689,-0.102596,0.20696,0.097699,0.532822,0.035472,109.784286


Under - Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions





Numbere of Fraudulent Transaction is 492

In [25]:
legit_sample = legit.sample(n=492)

Concatenating two DataFrames

In [26]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [27]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
86,55,-0.773450,0.853112,0.818254,-0.236070,0.803463,-1.438728,0.799479,-0.007989,-0.761090,...,0.035362,-0.116890,-0.178926,0.400155,-0.026231,0.165156,0.027762,0.132980,0.76,0.0
5745,6092,-1.048587,0.899177,2.488911,0.201191,-0.300900,0.103546,-0.096744,0.202863,1.876991,...,-0.311043,-0.335932,-0.299486,-0.157590,0.309509,0.367028,0.385191,0.188772,7.87,0.0
50848,44673,-0.339741,1.149495,1.302433,0.060004,0.056230,-0.985470,0.715795,-0.084107,-0.425411,...,-0.262419,-0.672120,-0.022079,0.340221,-0.154449,0.071231,0.247692,0.098646,0.89,0.0
94772,65023,-1.540715,1.575182,-0.224569,1.041269,0.048300,-0.830932,0.262160,0.661573,-0.295118,...,-0.017627,0.108239,0.099999,-0.067347,-0.229928,-0.340178,0.043305,-0.085648,19.12,0.0
38847,39578,-0.752845,1.045901,0.890107,0.935372,0.140186,-0.433494,0.323012,0.393536,-0.858277,...,0.255480,0.609056,-0.063802,0.091658,-0.297356,-0.305996,0.066874,0.064944,18.40,0.0


In [28]:
new_dataset.tail()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
92777,64093,-6.133987,2.941499,-5.593986,3.258845,-5.315512,-0.637328,-4.476488,1.695994,-1.606743,...,0.868340,0.793736,0.217347,-0.021985,0.145882,0.665088,-1.684186,0.310195,294.90,1.0
93424,64412,-1.348042,2.522821,-0.782432,4.083047,-0.662280,-0.598776,-1.943552,-0.329579,-1.853274,...,1.079871,-0.352026,-0.218358,0.125866,-0.074180,0.179116,0.612580,0.234206,1.00,1.0
93486,64443,1.079524,0.872988,-0.303850,2.755369,0.301688,-0.350284,-0.042848,0.246625,-0.779176,...,-0.023255,-0.158601,-0.038806,-0.060327,0.358339,0.076984,0.018936,0.060574,0.00,1.0
93788,64585,1.080433,0.962831,-0.278065,2.743318,0.412364,-0.320778,0.041290,0.176170,-0.966952,...,-0.008996,-0.057036,-0.053692,-0.026373,0.400300,0.072828,0.027043,0.063238,0.00,1.0
94218,64785,-8.744415,-3.420468,-4.850575,6.606846,-2.800546,0.105512,-3.269801,0.940378,-2.558691,...,0.102913,0.311626,-4.129195,0.034639,-1.133631,0.272265,1.841307,-1.796363,720.38,1.0


In [29]:
new_dataset['Class'].value_counts()

,count
Class,
0.0,492
1.0,217


In [30]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,40389.132114,-0.201232,-0.070129,0.728321,0.087316,-0.291074,0.175446,-0.100706,-0.009746,-0.111128,...,0.076743,-0.041832,-0.106820,-0.033419,0.032150,0.149419,-0.006949,-0.000812,0.009584,104.308638
1.0,35870.354839,-6.166180,4.217965,-8.087819,4.996030,-4.444345,-1.845983,-6.417590,2.796291,-2.953535,...,0.354728,0.731128,-0.127758,-0.247689,-0.102596,0.206960,0.097699,0.532822,0.035472,109.784286


Splitting the data into Features & Targets

In [31]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [32]:
print(X)

        Time        V1        V2        V3        V4        V5        V6  \
86        55 -0.773450  0.853112  0.818254 -0.236070  0.803463 -1.438728   
5745    6092 -1.048587  0.899177  2.488911  0.201191 -0.300900  0.103546   
50848  44673 -0.339741  1.149495  1.302433  0.060004  0.056230 -0.985470   
94772  65023 -1.540715  1.575182 -0.224569  1.041269  0.048300 -0.830932   
38847  39578 -0.752845  1.045901  0.890107  0.935372  0.140186 -0.433494   
...      ...       ...       ...       ...       ...       ...       ...   
92777  64093 -6.133987  2.941499 -5.593986  3.258845 -5.315512 -0.637328   
93424  64412 -1.348042  2.522821 -0.782432  4.083047 -0.662280 -0.598776   
93486  64443  1.079524  0.872988 -0.303850  2.755369  0.301688 -0.350284   
93788  64585  1.080433  0.962831 -0.278065  2.743318  0.412364 -0.320778   
94218  64785 -8.744415 -3.420468 -4.850575  6.606846 -2.800546  0.105512   

             V7        V8        V9  ...       V20       V21       V22  \
86     0.7994

In [33]:
print(Y)

86       0.0
5745     0.0
50848    0.0
94772    0.0
38847    0.0
        ... 
92777    1.0
93424    1.0
93486    1.0
93788    1.0
94218    1.0
Name: Class, Length: 709, dtype: float64


In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [35]:
print(X.shape, X_train.shape, X_test.shape)

(709, 30) (567, 30) (142, 30)


Model Training - Logistic Regression

In [36]:
model = LogisticRegression()

In [37]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Model Evalution

Accuracy Score

In [38]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [39]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9735449735449735


In [40]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [41]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9507042253521126


# Task
Install Streamlit, save the trained Logistic Regression model to a file, and then create a Streamlit application (`app.py`) to load the model, take user input for features, make predictions, and display the results. Finally, run the Streamlit application within Colab and expose it via ngrok to get a public URL for interaction.

## Install Streamlit and save the model

### Subtask:
Install the Streamlit library and save the trained Logistic Regression model to a file, so it can be loaded by the Streamlit application.


**Reasoning**:
The first instruction is to install the Streamlit library using `!pip install streamlit`.



In [42]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.8 MB/s eta 0:00:00


**Reasoning**:
The Streamlit library has been successfully installed. Now, I need to import the `pickle` module and save the trained Logistic Regression model to a file as instructed.



In [ ]:
import pickle

# Save the trained model
pickle.dump(model, open('logistic_regression_model.sav', 'wb'))
print("Model saved to 'logistic_regression_model.sav'")